In [ ]:
!pip -q install langchain openai tiktoken chromadb
!pip install langchain-community
!pip install pypdf
!pip install -qU langchain-cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.0/603.0 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.3 MB/s eta 0:00:

In [ ]:
!pip show langchain

Name: langchain
Version: 0.3.3
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community


## Setting up LangChain


In [ ]:
from langchain.vectorstores import Chroma

## Load multiple and process documents

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
loader = PyPDFLoader("/content/iesc111.pdf")
docs = loader.load()
text_splitter = CharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=50
    )
docs_split = text_splitter.split_documents(docs)

In [ ]:
# prompt: how to find number of chunks created in above line of code ?

print(len(docs_split))


14


In [ ]:
docs_split

[Document(metadata={'source': '/content/iesc111.pdf', 'page': 0}, page_content='Everyday we hear sounds from various\nsources like humans, bir ds, bells, machines,\nvehicles, televisions, radios etc. Sound is a\nform of ener gy which pr oduces a sensation\nof hearing in our ears. Ther e are also other\nforms of energy like mechanical energy, light\nenergy, etc. W e have talked about mechanical\nenergy in the pr evious chapters. Y ou have\nbeen taught about conservation of energy,\nwhich states that we can neither create nor\ndestr oy ener gy. W e  can just change it fr om\none for m to another . When you clap, a sound\nis produced. Can you produce sound without\nutilising your energy? Which form of energy\ndid you use to produce sound? In this\nchapter we are going to learn how sound is\nproduced and how it is transmitted through\na medium and received by our ears.\n11.1 Production of Sound\nActivity _____________ 11.1\n•Take a tuning fork and set it vibrating\nby striking its pr ong o

## create the DB

In [ ]:
import os
os.environ["COHERE_API_KEY"]='Ox97SolGnL68xrDjbNAMiVaWCqZ5Fny3d7hYAub6'

In [ ]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(model="embed-english-v3.0")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [ ]:
persist_directory = 'Physics'
vectordb = Chroma.from_documents(documents=docs_split, embedding=embeddings, persist_directory=persist_directory)

## Make a retriever

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.invoke("how does sound propagate")

In [ ]:
print(len(docs))
print(docs[1])

4
page_content='SCIENCE 128the vibrating object to the ear . A particle of
the medium in contact with the vibrating
object is first displaced fr om its equilibrium
position. It then exerts a for ce on the adjacent
particle. As a r esult of which the adjacent
particle gets displaced fr om its position of
rest. After displacing the adjacent particle the
first particle comes back to its original
position. This pr ocess continues in the
medium till the sound r eaches your ear . The
disturbance created by a source of  sound in
the medium travels through the medium and
not the particles of the medium.
A wave is a disturbance that moves
through a medium when the particles of the
medium set neighbouring particles into
motion. They in tur n produce similar motion
in others. The particles of the medium do not
move forwar d themselves, but the
disturbance is carried forwar d. This is what
happens during pr opagation of sound in a
medium, hence sound can be visualised as a
wave. Sound waves are ch

In [ ]:
print(retriever.search_type)
print(retriever.search_kwargs)

similarity
{}


In [ ]:
from langchain_cohere import ChatCohere, CohereRagRetriever
from langchain_core.documents import Document
llm=ChatCohere()

## Make a chain

In [ ]:
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

# Create a RAG chain with the retriever and the language model
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff" refers to simple stuffing of retrieved documents
    retriever=retriever
)

# Define a tool that uses the RAG chain to handle queries
rag_tool = Tool(
    name="DocumentQA",
    func=rag_chain.run,
    description="Use this tool to answer questions by retrieving and using context from documents."
)

# Initialize an agent with the RAG tool
agent = initialize_agent(
    tools=[rag_tool],
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # This agent type can decide which tool to use based on the description
    llm=llm,
    verbose=True
)

<ipython-input-18-3f90c35fe41d>:20: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [ ]:
def query_rag_chain(query):
  """Queries the RAG chain with the given query and prints the response."""

  response = agent.run(query)
  print(response)

query = input("Enter your question: ")
query_rag_chain(query)

while True:
  query = input("Enter your question (or type 'exit' to quit): ")
  if query.lower() == 'exit':
    break
  query_rag_chain(query)

Enter your question: hi


<ipython-input-19-416d852a21cc>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)




> Entering new AgentExecutor chain...
Thought: This is a greeting. I will respond in kind.
Final Answer: Hello! How can I assist you today?

> Finished chain.
Hello! How can I assist you today?
Enter your question (or type 'exit' to quit): what is sound ?


> Entering new AgentExecutor chain...
Thought: To answer this question, I need to provide a definition of sound.
Action: DocumentQA
Action Input: What is sound?
Observation:
Observation: Sound is a form of energy that produces a sensation of hearing in our ears. It is produced by vibrating objects, which set the particles of the surrounding medium (solid, liquid, or gas) into motion. This motion creates a series of compressions and rarefactions in the medium, resulting in the propagation of sound waves.
Thought:Thought: The observation provides a comprehensive explanation of sound, covering its nature, production, and propagation.
Final Answer: Sound is a form of energy that creates a sensation of hearing. It is generated by vibra

In [ ]:
API_KEY = 'b1afee3b-c36c-4abf-8c35-5aeec8cba897'

In [ ]:
!pip install sounddevice scipy requests-toolbelt


In [ ]:
! pip install git+https://github.com/openai/whisper.git
! pip install sounddevice wavio
! pip install ipywebrtc notebook

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-60jbcaxd
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-60jbcaxd
  Resolved https://github.com/openai/whisper.git to commit 25639fc17ddc013d56c594bfbf7644f2185fad84
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803321 sha256=a2218f98c5e68115591ee7beed8da53dbc65aef8ed76f1f6b70b08c60408e997
  Stored in directory: /tmp/pip-ephem-wheel-cache-w5y_d9ke/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 9.3 MB/s eta 0:00:00
Using cach

In [ ]:
!apt install ffmpeg
!apt-get install libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 65.3 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Fetched 65.3 kB in 0s (1,113 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123621 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Setting up libportaudio2:amd64 (19.6.0-1.1) ...
Processing triggers fo

In [ ]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio

from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio, display
import ipywidgets as widgets

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [34]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [35]:
recorder.save('test.wav')


!ffmpeg -i /content/2.webm /content/2.wav

In [36]:
import requests
import os

os.environ['API_KEY'] = "b1afee3b-c36c-4abf-8c35-5aeec8cba897"

# Load your .wav file
with open('/content/test.wav', 'rb') as audio_file:
    files = {
        'file': ('test.wav', audio_file, 'audio/wav'),  # Specify the filename and content type
    }
    data = {
        'prompt': '<string>',
        'model': 'saaras:v1'
    }

    url = "https://api.sarvam.ai/speech-to-text-translate"
    headers = {
        "API-Subscription-Key": os.environ['API_KEY'],
    }

    response = requests.post(url, files=files, data=data, headers=headers)

print(response.text)


{"transcript":"Hi, what is sound?","language_code":null}


In [37]:
import json

# Assuming response.text contains the JSON string
data = json.loads(response.text)

# Now you can extract the transcript
transcript = data["transcript"]
transcript

'Hi, what is sound?'

In [ ]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.5 MB/s eta 0:00:00


In [38]:
import streamlit as st

# Streamlit interface
st.title("Smart RAG Agent with Voice Integration")

# Text input for the user query
query = st.text_input("Enter your question:")

if st.button("Submit"):
    if query:
      try:
        # Get agent's response
        response = query_rag_chain(query)
        st.write(f"Response: {response}")

      except Exception as e:
        st.error(f"An error occurred: {e}")

ModuleNotFoundError: No module named 'streamlit'